In [1]:
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer

import torch
from transformers import AutoTokenizer, AutoModel
import plotly.express as px
import datetime

# Генерация эмбеддингов для строк

In [2]:
print(datetime.datetime.now())

2022-03-09 01:43:53.519274


## Загрузка данных

In [19]:
df_education = pd.read_csv('data/uuid_x_education_fields.csv', index_col=0)

In [20]:
df_education

,org_uuid,education_school_name,education_field_of_study,education_description,education_degree,education_start_date_year,education_end_date,education_school_link
0,438c7e89-ba0c-b8cc-7ea6-1a0ac47ae40d,Davidson College,English,NaN,BA,NaN,NaN,https://www.linkedin.com/school/davidson-college/
0,438c7e89-ba0c-b8cc-7ea6-1a0ac47ae40d,Queen Mary University of London,"English, History",Semester study abroad experience.,NaN,NaN,NaN,https://www.linkedin.com/school/queen-mary-uni...
0,b37187f2-8ad4-8225-cfa2-2757f2ef9bc4,Kyungpook National University,"Electrical, Electronics and Communications Eng...",NaN,Bachelor of Science (BS),1985.0,1989,https://www.linkedin.com/school/%EA%B2%BD%EB%B...
0,75e76c56-3026-404d-079d-be335a63d9ff,Highline College,NaN,"Undergraduate Studies, One Year \nTechnology S...",NaN,NaN,NaN,https://www.linkedin.com/school/highline-college/
0,75e76c56-3026-404d-079d-be335a63d9ff,Highline Community Collage,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0,3a372b39-86d8-a24e-9fea-3be65013b476,Harpur College,Political Behavior,NaN,Bachelor of Arts - BA,1959.0,1963,NaN
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,University of Florida College of Medicine,NaN,NaN,Doctor of Medicine (M.D.),NaN,NaN,https://www.linkedin.com/school/university-of-...
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,University of Virginia,NaN,NaN,Bachelor's degree,NaN,NaN,https://www.linkedin.com/school/university-of-...
0,fcc72760-5eb8-a34e-718a-5f0e02ec916f,Yale School of Management,NaN,NaN,MBA,NaN,NaN,https://www.linkedin.com/school/yale-school-of...


## LaBSE

https://huggingface.co/cointegrated/LaBSE-en-ru

In [13]:
def get_education_enbeddings(df_education):
    
    tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
    model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

    embedding_vec = []
    df_education_study = df_education[['education_field_of_study', 'education_school_name']].drop_duplicates().reset_index(drop=True)

    for i, row in enumerate(df_education_study['education_field_of_study']):
        
        print('{} / {}'.format(i+1, df_education_study.shape[0]), end="\r")

        embeddings = []

        if row == row:
            encoded_input = tokenizer(row, padding=True, truncation=True, max_length=64, return_tensors='pt')

            with torch.no_grad():
                model_output = model(**encoded_input)

            embeddings = model_output.pooler_output
            embeddings = torch.nn.functional.normalize(embeddings)

        embedding_vec.extend(np.array(embeddings))

    df_embeddings = pd.DataFrame(embedding_vec)
    
    df_embeddings = pd.concat(
        [
            df_embeddings, 
            df_education_study
        ], axis=1
    )
    
    return df_embeddings

In [14]:
df_embeddings = get_education_enbeddings(df_education)
#df_embeddings.to_csv('data/embeddings/education_field_of_study.csv')

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
df_embeddings

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,education_field_of_study,education_school_name
0,-0.031753,-0.014959,-0.023481,-0.042621,-0.029021,0.038917,-0.057827,0.019492,-0.076168,-0.064734,...,0.031098,-0.017985,0.047823,0.027166,-0.032544,-0.009658,-0.049922,-0.014915,English,Davidson College
1,-0.060066,-0.062174,-0.017558,-0.027535,-0.058638,0.032005,-0.036692,-0.047136,-0.025333,-0.019963,...,-0.020289,-0.047562,0.033319,0.003101,-0.054761,0.024514,-0.061725,-0.001204,"English, History",Queen Mary University of London
2,-0.006098,-0.062554,-0.048228,0.004714,-0.053709,-0.014195,-0.044954,-0.003859,-0.035358,-0.041296,...,-0.015792,-0.060373,0.039580,0.020569,-0.060297,-0.029275,0.006168,-0.010828,"Electrical, Electronics and Communications Eng...",Kyungpook National University
3,-0.041572,-0.048041,-0.032339,-0.049845,-0.033356,0.065038,-0.040907,-0.035030,-0.035901,-0.003692,...,0.027216,-0.075004,0.033958,-0.063679,-0.023682,-0.012145,0.004746,-0.030422,NaN,Highline College
4,-0.019766,-0.066109,-0.019090,-0.013277,-0.069746,-0.008724,-0.045727,-0.015144,0.002470,0.003608,...,0.007296,-0.021445,-0.049503,-0.046420,-0.062146,0.022783,-0.002651,-0.029784,NaN,Highline Community Collage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chemistry,Columbia University in the City of New York
2993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physical Chemistry,University of Science and Technology of China
2994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St. John's University
2995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Political Behavior,Harpur College


In [15]:
#df_embeddings = pd.read_csv('data/embeddings/education_field_of_study.csv', index_col=0)
vec_embedding = df_embeddings.loc[:, ~df_embeddings.columns.isin(
    ['education_field_of_study', 'education_school_name']
)]

In [16]:
vec_embedding

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.031753,-0.014959,-0.023481,-0.042621,-0.029021,0.038917,-0.057827,0.019492,-0.076168,-0.064734,...,0.010087,-0.004319,0.031098,-0.017985,0.047823,0.027166,-0.032544,-0.009658,-0.049922,-0.014915
1,-0.060066,-0.062174,-0.017558,-0.027535,-0.058638,0.032005,-0.036692,-0.047136,-0.025333,-0.019963,...,0.002764,-0.050229,-0.020289,-0.047562,0.033319,0.003101,-0.054761,0.024514,-0.061725,-0.001204
2,-0.006098,-0.062554,-0.048228,0.004714,-0.053709,-0.014195,-0.044954,-0.003859,-0.035358,-0.041296,...,-0.061741,-0.008394,-0.015792,-0.060373,0.039580,0.020569,-0.060297,-0.029275,0.006168,-0.010828
3,-0.041572,-0.048041,-0.032339,-0.049845,-0.033356,0.065038,-0.040907,-0.035030,-0.035901,-0.003692,...,-0.063321,-0.000296,0.027216,-0.075004,0.033958,-0.063679,-0.023682,-0.012145,0.004746,-0.030422
4,-0.019766,-0.066109,-0.019090,-0.013277,-0.069746,-0.008724,-0.045727,-0.015144,0.002470,0.003608,...,-0.037422,-0.017773,0.007296,-0.021445,-0.049503,-0.046420,-0.062146,0.022783,-0.002651,-0.029784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
